In [1]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
import os
import brunoflow as bf
import jax
from jax import numpy as jnp
import numpy as np
import torch
import unittest as ut
print(f"Running JAX on {jax.devices()[0].device_kind}")

env: XLA_PYTHON_CLIENT_PREALLOCATE=false
Running JAX on NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [2]:
from brunoflow.test import utils
inputs = utils.random_inputs([[(6, 5)]])
fn=lambda x: x[1:3, 1]
torch_fn=lambda x: x[1:3, 1]

In [3]:
inputs

In [4]:
for inp in inputs:
    x = utils.inputs_to_params(inp)
    x_t = utils.inputs_to_torch(inp)
    y = fn(*x)
    y_t = torch_fn(*x_t)
    utils.check(ut.TestCase(), y, y_t)

In [5]:
import numpy as np
a = bf.Parameter(np.ones((10, 10)))
np.zeros_like(a.val)

# a

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
x, x_t

([node(name: None, val: [[-0.45614302  1.92889087 -0.81652471  1.26533222  0.41356366]
   [-0.65349289 -0.76481893 -0.19027144  0.28957672 -1.15048079]
   [-0.14026162 -0.1134748   0.89817177 -0.37637173 -0.62206724]
   [-1.30590398 -0.54389701 -2.14626584 -0.07253473  0.49106049]
   [ 0.9034091   0.87678393  1.60258579  1.50054879 -1.07206867]
   [-1.12693568  0.98924257 -2.08546269  0.41695462 -0.81677451]], grad: [[0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]])],
 [tensor([[-0.4561,  1.9289, -0.8165,  1.2653,  0.4136],
          [-0.6535, -0.7648, -0.1903,  0.2896, -1.1505],
          [-0.1403, -0.1135,  0.8982, -0.3764, -0.6221],
          [-1.3059, -0.5439, -2.1463, -0.0725,  0.4911],
          [ 0.9034,  0.8768,  1.6026,  1.5005, -1.0721],
          [-1.1269,  0.9892, -2.0855,  0.4170, -0.8168]], dtype=torch.float64)])

In [2]:
import torch

def _einsum_forward(subscripts: str, arr1, arr2):
    """Function to compute the einsum of two tensors of arbitrary shape."""
    return jnp.einsum(subscripts, arr1, arr2)

def jax_einsum_backward_func(einsum_expr, A, B):
    jac_A, jac_B = jax.jacfwd(_einsum_forward, argnums=(1, 2))(einsum_expr, A, B)
    # print(jac_A.shape)
    # print(jac_B.shape)
    # print(jnp.einsum("ijk,ik->jk", jac_A, jnp.ones_like(_einsum_forward(einsum_expr, A, B))))
    A_grad = jnp.sum(jac_A, axis=tuple(range(len(jac_A.shape) - len(A.shape))))
    B_grad = jnp.sum(jac_B, axis=tuple(range(len(jac_B.shape) - len(B.shape))))
    
    return A_grad, B_grad

def check_backward_einsum(einsum_expr, A, B, jax_einsum_backward_func):
    torch_A = torch.tensor(np.array(A), requires_grad=True)
    torch_B = torch.tensor(np.array(B), requires_grad=True)
    out = torch.einsum(einsum_expr, torch_A, torch_B)
    out.backward(torch.ones_like(out))

    A_jax_grad, B_jax_grad = jax_einsum_backward_func(einsum_expr, A, B)
    
    assert np.array_equal(A_jax_grad, torch_A.grad), f"A_jax_grad value does not equal expected torch_A.grad value.\nExpected: {torch_A.grad}\nActual:{A_jax_grad}"
    assert np.array_equal(B_jax_grad, torch_B.grad), f"B_jax_grad value does not equal expected torch_B.grad value.\nExpected: {torch_B.grad}\nActual:{B_jax_grad}"



In [3]:
A = jnp.ones(shape=(5, 10))
B = jnp.ones(shape=(10, 2))
einsum_expr = "ij,jk->ik"
check_backward_einsum(einsum_expr, A, B, jax_einsum_backward_func)

In [ ]:
A = jnp.ones(shape=(5, 10))
B = jnp.ones(shape=(10, 2))
einsum_expr = "ij,jk->i"
check_backward_einsum(einsum_expr, A, B, jax_einsum_backward_func)

In [ ]:
A = jnp.ones(shape=(5, 10, 2))
B = jnp.ones(shape=(10, 2, 3, 3))
einsum_expr = "ijk,jkll->ikl"
check_backward_einsum(einsum_expr, A, B, jax_einsum_backward_func)


In [ ]:
b, h, l, r, d = 3, 2, 1, 9, 7
A = np.random.normal(size=(b, h, l, d))
B = np.random.normal(size=(l, r, d))
einsum_expr = "bhld,lrd->bhlr"
check_backward_einsum(einsum_expr, A, B, jax_einsum_backward_func)


In [ ]:
A = jnp.ones(shape=(5, 10))
B = jnp.ones(shape=(10, 2))
C = jnp.ones(shape=(2, 4))
einsum_expr1 = "ij,jk->ik"
einsum_expr2 = "ij,jk->i"

def chained_einsum(A, B, C, einsum_expr1, einsum_expr2):
    return _einsum_forward(einsum_expr2, _einsum_forward(einsum_expr1, A, B), C)

jac_A, jac_B = jax.jacfwd(chained_einsum, argnums=(0, 1))(A, B, C, einsum_expr1, einsum_expr2)
A_jax_grad = jnp.sum(jac_A, axis=tuple(range(len(jac_A.shape) - len(A.shape))))
B_jax_grad = jnp.sum(jac_B, axis=tuple(range(len(jac_B.shape) - len(B.shape))))

torch_A = torch.tensor(np.array(A), requires_grad=True)
torch_B = torch.tensor(np.array(B), requires_grad=True)
torch_C = torch.tensor(np.array(C), requires_grad=True)
out = torch.einsum(einsum_expr2, torch.einsum(einsum_expr1, torch_A, torch_B), torch_C)
out.backward(torch.ones_like(out))

assert np.array_equal(A_jax_grad, torch_A.grad)
assert np.array_equal(B_jax_grad, torch_B.grad)
# check_backward_einsum(einsum_expr, A, B, jax_einsum_backward_func)

NameError: name '_einsum_forward' is not defined

In [18]:
X = jnp.ones(shape=(5, 10))
W = jnp.ones(shape=(10, 1))
y = jnp.ones(shape=(5, 1))
einsum_expr = "ij,jk->ik"

bf_X = bf.Node(X, name="X")
bf_W = bf.Node(W, name="W")
logits = bf.einsum("ij,jk->ik", bf_X, bf_W)
loss = bf.opt.mse_loss(logits, y)
loss.backprop(values_to_compute=("grad", "abs_val_grad", "entropy"))
# print(bf_X.grad)

bf_X_mm = bf.Node(X.copy(), name="X_mm")
bf_W_mm = bf.Node(W.copy(), name="W_mm")
logits = bf.matmul(bf_X_mm, bf_W_mm)
loss = bf.opt.mse_loss(logits, y)
loss.backprop(values_to_compute=("grad", "abs_val_grad", "entropy"))
# print(bf_X_mm.grad)

assert np.array_equal(bf_X.grad, bf_X_mm.grad)
assert np.array_equal(bf_X.abs_val_grad, bf_X_mm.abs_val_grad)
assert np.array_equal(bf_X.entropy_wrt_output, bf_X_mm.entropy_wrt_output)

# torch_X = torch.tensor(np.array(X), requires_grad=True)
# torch_W = torch.tensor(np.array(W), requires_grad=True)
# torch_y = torch.tensor(np.array(y), requires_grad=True)
# out = torch.nn.MSELoss()(torch.einsum(einsum_expr1, torch_X, torch_W), torch_y)
# out.backward()
# print(torch_X.grad)
# # assert np.array_equal(A_jax_grad, torch_A.grad)
# # assert np.array_equal(B_jax_grad, torch_B.grad)
# # check_backward_einsum(einsum_expr, A, B, jax_einsum_backward_func)

AssertionError: 

In [7]:
# x = jnp.ones(shape=(5, 2))
# out = bf.gelu(x)
# out.backprop()

x_torch = torch.tensor(np.array(x), requires_grad=True)
out = torch.nn.functional.gelu(x_torch)
out.backward(torch.ones_like(out))
print(x_torch.grad)

tensor([[1.0833, 1.0833],
        [1.0833, 1.0833],
        [1.0833, 1.0833],
        [1.0833, 1.0833],
        [1.0833, 1.0833]])


In [13]:
X = jnp.ones(shape=(5, 2))
jac = jax.jacfwd(lambda x: jax.nn.gelu(x, False))(X)
jnp.sum(jac, axis=tuple(range(len(jac.shape) - len(X.shape))))

DeviceArray([[1.0833155, 1.0833155],
             [1.0833155, 1.0833155],
             [1.0833155, 1.0833155],
             [1.0833155, 1.0833155],
             [1.0833155, 1.0833155]], dtype=float32)